# 스레드

https://docs.python.org/3.7/library/threading.html

파이썬에서 멀티스레드의 구현은 threading 모듈을 이용합니다.

In [2]:
# 음식 배달과 그릇 찾기 2가지 작업을 순차적으로 수행하는 코드

class Delivery:
	def run(self):
		print("delivery")

class RetriveDish:
	def run(self):
		print("Retriving Dish")

work1 = Delivery()
work2 = RetriveDish()

def main():
	work1.run()
	work2.run()

if __name__ == '__main__':
    main()

delivery
Retriving Dish


- threading 모듈을 import하고
- 클래스에 Thread를 상속받습니다.

In [3]:
from threading import *

class Delivery(Thread):
	def run(self):
		print("delivery")

class RetriveDish(Thread):
	def run(self):
		print("Retriving Dish")

work1 = Delivery()
work2 = RetriveDish()

def main():
	work1.run()
	work2.run()

if __name__ == '__main__':
    main()

delivery
Retriving Dish


In [5]:
from threading import *

class Delivery:
    def run(self):
        print("delivering")

work1 = Delivery()
print(work1.run)

class Delivery(Thread):
    def run(self):
        print("delivering")

work2 = Delivery()
print(work2.run)
# work2 는 다른 스레드에 할당

<bound method Delivery.run of <__main__.Delivery object at 0x00000253E2BFA4C8>>
<bound method Delivery.run of <Delivery(Thread-10, initial)>>


threading 모듈의 Thread 클래스를 상속받아서 구현할 수도 있지만 그대로 인스턴스화하여 스레드를 생성할 수도 있습니다.

인스턴스화 하려면 Thread 클래스에 인자로 target과 args 값을 넣어 줍니다. args에 넣어 준 파라미터는 스레드 함수의 인자로 넘어갑니다.

t = Thread(target=함수이름, args=())

Thread 클래스에는 start(), join() 같은 스레드 동작 관련 메소드가 있습니다. Thread로 실행할 함수를 정의한 후 start() 를 통해 스레드를 실행합니다.

In [6]:
from threading import *
from time import sleep

Stopped = False

def worker(work, sleep_sec):    # 일꾼 스레드입니다.
    while not Stopped:          # 그만 하라고 할때까지
        print('do ', work)      # 시키는 일을 하고
        sleep(sleep_sec)        # 잠깐 쉽니다.
    print('retired..')          # 언젠가 이 굴레를 벗어나면, 은퇴할 때가 오겠지요?
        
t = Thread(target=worker, args=('Overwork', 3))    # 일꾼 스레드를 하나 생성합니다. 열심히 일하고 3초간 쉽니다.
t.start()    # 일꾼, 이제 일을 해야지? 😈

# 멈추지 않으면 계속 함

do  Overwork
do  Overwork
do  Overwork
do  Overwork
do  Overwork
do  Overwork


이 일꾼의 일을 어떻게 멈춰주면 좋을까요? t.terminate() 같은 메소드가 있느냐 하면, 없습니다.

그래서 스레드 함수가 루프를 돌 때는 꼭 멈춰야 할지를 체크하는 flag(여기서는 Stopped)를 체크하도록 설계해야 합니다

In [7]:
# 이 코드 블럭을 실행하기 전까지는 일꾼 스레드는 종료하지 않습니다. 
Stopped = True    # 일꾼 일 그만하라고 세팅해 줍시다. 
t.join()          # 일꾼 스레드가 종료할때까지 기다립니다. 
print('worker is gone.')

retired..
worker is gone.


# 멀티프로세스

파이썬에서 멀티프로세스의 구현은 multiprocessing 모듈을 이용해서 할 수 있습니다.

https://docs.python.org/3.7/library/multiprocessing.html

Process 인스턴스를 만든 뒤, target 과 args 파라미터에 각각 함수 이름과 함수 인자를 전달합니다.

In [8]:
import multiprocessing as mp

def delivery():
    print('delivering...')

p = mp.Process(target=delivery, args=())
p.start()

In [9]:
p = mp.Process(target=delivery, args=())
p.start() # 프로세스 시작
p.join() # 실제 종료까지 기다림 (필요시에만 사용)
p.terminate() # 프로세스 종료

지금까지 스레드와 프로세스를 생성하는 방법을 배웠습니다. 사실 멀티스레드/프로세스 작업을 할 때 가장 많은 연산이 필요한 작업은 스레드나 프로세스를 생성하고 종료하는 일입니다. 특히 스레드/프로세스를 사용한 뒤에는 제대로 종료해 주어야 컴퓨팅 리소스가 낭비되지 않습니다.

또 지금까지 본 것처럼 하나씩 하나씩 실행한다고 전체적인 프로그램의 성능이 좋아지지는 않아요. 오히려 더 번거로울 수 있습니다. 그래서 실제로 사용할 때에는 스레드/프로세스 풀을 사용해서 생성합니다.

"풀(Pool)"은 스레드나 프로세스들로 가득 찬 풀장이라고 생각하면 됩니다. 스레드 풀을 만들면 각각의 태스크들에 대해 자동으로 스레드들을 할당하고 종료합니다.

풀을 만드는 방법은 크게 2가지가 있습니다.

- Queue를 사용해서 직접 만드는 방법

- concurrent.futures 라이브러리의 ThreadPoolExecutor , ProcessPoolExecutor클래스를 이용하는 방법

2가지 방법은 모두 많이 사용됩니다. 아직 자료구조에 대해 배우지 않았으니 여기서는 concurrent.futures 모듈을 사용하는 방법만을 이용해 구현해 보겠습니다.

# concurrent.futures 모듈 소개
이 모듈은 파이썬 3.2부터 추가된 모듈입니다. Java 언어를 배운 적이 있다면 ThreadPoolExecutor 에 대해 들어본 적이 있을 거예요. 이를 파이썬에서 구현한 형태라고 생각하시면 편합니다. 한국말로 "동시성 퓨처"라고 번역해서 부르기도 하는데, 기능은 크게 4가지가 있습니다.

- Executor 객체
- ThreadPoolExecutor 객체
- ProcessPoolExecutor 객체
- Future 객체

여기서는 Thread/Process Pool 구현에 필요한 부분만 살펴보겠습니다.

# ThreadPoolExecutor
Executor 객체를 이용하면 스레드 생성, 시작, 조인 같은 작업을 할 때, with 컨텍스트 관리자와 같은 방법으로 가독성 높은 코드를 구현할 수 있습니다. (프로세스 구현 방법 역시 동일하므로 설명은 스레드로만 하겠습니다.)

In [10]:
from concurrent.futures import ThreadPoolExecutor

class Delivery:
    def run(self):
        print("delivering")
w = Delivery()

with ThreadPoolExecutor() as executor:
    future = executor.submit(w.run)

delivering


# multiprocessing.Pool
multiprocessing.Pool.map을 통해 여러 개의 프로세스에 특정 함수를 매핑해서 병렬처리하도록 구현하는 방법이 널리 사용됩니다.

In [ ]:
from multiprocessing import Pool
from os import getpid

def double(i):
    print("I'm processing ", getpid())    # pool 안에서 이 메소드가 실행될 때 pid를 확인해 봅시다.
    return i * 2

with Pool() as pool:
      result = pool.map(double, [1, 2, 3, 4, 5])
      print(result)

위에서 double(i)이라는 메소드가 pool을 통해 각각 다른 pid를 가진 프로세스들 위에서 multiprocess로 실행되었다는 것을 확인할 수 있습니다.